https://www.med66.com/sitemap/yixuetiku/1.htm

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

# 解析

In [2]:
def get_md5(file_name):
    hash = hashlib.md5()
    hash.update(file_name.encode('gb18030'))
    return hash.hexdigest()

def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            fullname = os.path.join(root, f)
            yield fullname

In [37]:
def parser(html_file):
    html_content = open(html_file, 'r', encoding='gb18030').read()
    selector = Selector(text=html_content)
    questionlist = selector.xpath("//h1/text()").extract()
    contentformat = ['//div[@id="fontzoom"]//text()', '//div[@class="content"]//text()', '//font[@id="zoom"]//text()']
    for i in contentformat:
        fullcontent = selector.xpath(i).extract()
        if len(fullcontent) != 0:
            break
    if len(fullcontent) == 0:
        print("可能还有其他格式")
        print(html_file)
        return None
    # fullcontent = selector.xpath("//div[@id='fontzoom']//text()").extract()
#     print(len(fullcontent))
#     if len(fullcontent) == 0:
#         fullcontent = selector.xpath("//div[@class='content']//text()").extract()
# #         print("111")
# #         fullcontent = selector.xpath("//div[@class='content']").xpath('string(.)').extract()[0]
#         if len(fullcontent) == 0:
#             print("可能还有其他格式")
#             print(html_file)
#             return None
    # print(fullcontent)
    # 如果在fullcontent里同时有【选项】和【答案】字段
    # -》提取【选项】和【答案】
    # TODO: 有些用中文冒号，有些用英文，不如先不加冒号作为分隔符，之后再用strip删除
    questionindex = 999
    optionindex = 999
    answerindex = 999
    analysisindex = 999
    
    for index, item in enumerate(fullcontent):
        if item.startswith("原题"):
            questionindex = index # 获取原题开始的的索引
        if item.startswith("选项"):
            optionindex = index
        if item.startswith("答案"):
            answerindex = index
        if item.startswith("解析"):
            analysisindex = index
    if optionindex != 999 and answerindex != 999:
        if questionindex != 999:
            questionlist = fullcontent[questionindex+1:optionindex]
            optionslist = fullcontent[optionindex+1:answerindex]
            answerlist = fullcontent[answerindex+1:]
            if analysisindex != 999:
                answerlist = fullcontent[answerindex+1:analysisindex]
                analysislist = fullcontent[analysisindex+1:]
            else:
                analysislist = ['无解析']
        else:
            optionslist = fullcontent[optionindex+1:answerindex]
            answerlist = fullcontent[answerindex+1:]
            if analysisindex != 999:
                answerlist = fullcontent[answerindex+1:analysisindex]
                analysislist = fullcontent[analysisindex+1:]
            else:
                analysislist = ['无解析']
    else:
        return None
    
    # 拼接字符串
    question = ''.join(questionlist)
    answer = ''.join(answerlist)
    analysis = ''.join(analysislist)
    options = []
    
    # "选项"不拼接成字符串
    for item in optionslist:
        options.append(''.join(item.split()))
#     print(options)
    
    # 删除特殊字符
    tmp = {
        'question': ''.join(question.split()),
        'options': options,
        'answer': ''.join(answer.split()),
        'analysis': ''.join(analysis.split())
    }
#     print(tmp)
    qa_results.append(tmp)
    
#     pbar.update(1)
    return None


In [38]:
# 先不用多进程试试
qa_results = []
count = 0
for html_file in findAllFile("D:/Intern_Project/13_医学题库/详情页"):
    count += 1
    parser(html_file)
    if count % 1000 == 0:
        print(count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
可能还有其他格式
D:/Intern_Project/13_医学题库/详情页\34483.html
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
可能还有其他格式
D:/Intern_Project/13_医学题库/详情页\65836.html
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000


In [39]:
with open('D:/Intern_Project/13_医学题库/qa_results.json', 'w', encoding='gb18030') as f:
    f.write(json.dumps(qa_results, indent=2, ensure_ascii=False))

In [40]:
print(len(qa_results))

66905


In [4]:
def task_monitor(task_list, mt_pool):
    count = 0
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
#                 if result is None:
#                     continue
#                 else：
#                     qa_results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [ ]:
if __name__ == "__main__":    
    qa_results = []
    all_craw_tasks = []
    pbar = tqdm(total=(116321-700))
    
    with ThreadPoolExecutor(max_workers=10) as mt_pool:
        for html_file in findAllFile("D:/Intern_Project/13_医学题库/详情页"):
            tmp_task = mt_pool.submit(parser, html_file)
            all_craw_tasks.append(tmp_task) 
        # 监测下载任务进展
        task_monitor(all_craw_tasks, mt_pool)
        

  0%|                                                                           | 51/115621 [01:43<72:48:18,  2.27s/it]

In [9]:
with open('D:/Intern_Project/13_医学题库/qa_results.json', 'w', encoding='gb18030') as f:
    f.write(json.dumps(qa_results, indent=2, ensure_ascii=False))